In [1]:
#https://towardsdatascience.com/graphframes-in-jupyter-a-practical-guide-9b3b346cebc5
from pyspark import *
from pyspark.sql import *
spark = SparkSession.builder.appName('fun').getOrCreate()
vertices = spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])
edges = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])
g = GraphFrame(vertices, edges)
## Take a look at the DataFrames
g.vertices.show()
g.edges.show()
## Check the number of edges of each vertex
g.degrees.show()

Exception: Java gateway process exited before sending its port number

In [2]:
#Take a look at the DataFrames
g.vertices.show()
g.edges.show()
g.degrees.show()
g.inDegrees.show()
g.outDegrees.show()

NameError: name 'g' is not defined

In [ ]:
numFollows=g.edges.filter("type='follows'").count()
print("The number of follow edges is", numFollows)

In [ ]:
g.vertices.filter("age>30").show()
g.inDegrees.filter("inDegrees>=2").sort("inDegrees", ascending=False).show()
g.edges.filter('type=="friend"').show()

In [ ]:
#search for pairs of vertices with edges in both directions between them
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)")
motifs.show()

In [ ]:
sc.setCheckpointDir('graphframes_cps')
g.connectedComponents().show()

In [ ]:
g.triangleCount().show()

In [ ]:
pr=g.pageRank(resetProbability=0.15, tol=0.01)  #look at the pagerank score for every vertices
pr.vertics.show()  #look at the weight of every edge
pr.edges.show()

## Path and Distance

In [ ]:
paths=g.bfs("name='Charlie'", "DQ=1", edgeFiler="relationship !='mtg'", maxPathLength=3)
paths.show()

In [ ]:
A left anti join returns rows from left dataset which do not have a match in the right dataset

In [ ]:
#starting from bad nodes
v_0 = v.select('id','name').filter('DQ=1').withColumn('DQ_dist',lit(0)).cache()
v_0.show()

#e_0 keeps track of the list of nodes that have not been explored,keep removing found nodes from it
e_0=e

#find 1st degree and 2nd degree connected
for i in range(1,3):
    e_0 = e_0.join (v_0.select('id'), expr('id = dst'), how='left_anti')
    
    #v_0 keeps the current nth degree connected. find next nodes from e_0 that are connected to current v_0
    
    v_0=e_0.join(v_0, expr('id = src')
                 ).select('dst').join(v, expr('id = dst')
                ).select('id','name').withColumn('DQ_dist', lit(i)
    ).distinct().cache()
    
    v_0.show()
